In [25]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var 

In [26]:
n_batch = 64
learning_rate = 0.01
n_epoch = 3
n_print = 10

In [7]:
T = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=T)
val_data = torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=T)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)
val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)

In [27]:
class myCNN(nn.Module):
  def __init__(self):
    super(myCNN,self).__init__()
    self.cnn1 = nn.Conv2d(1,3,3)
    self.cnn2 = nn.Conv2d(3,2,5)
    self.linear = nn.Linear(968,10)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    n = x.size(0)
    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = x.view(n,-1)
    x = self.linear(x)
    return x


In [28]:
mycnn = myCNN().cuda()
cec = nn.CrossEntropyLoss()
optimizer = optim.Adam(mycnn.parameters(),lr = learning_rate)

def validate(model,data):
  # To get validation accuracy = (correct/total)*100.
  total = 0
  correct = 0
  for i,(images,labels) in enumerate(data):
    images = var(images.cuda())
    x = model(images)
    value,pred = torch.max(x,1)
    pred = pred.data.cpu()
    total += x.size(0)
    correct += torch.sum(pred == labels)
  return correct*100./total

for e in range(n_epoch):
  for i,(images,labels) in enumerate(train_dl):
    images = var(images.cuda())
    labels = var(labels.cuda())
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn,val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data),'Accuracy :',accuracy,'%')

Epoch : 1 Batch : 10 Loss : 1.5697802305221558 Accuracy : 61.220001220703125 %
Epoch : 1 Batch : 20 Loss : 1.0018867254257202 Accuracy : 77.8499984741211 %
Epoch : 1 Batch : 30 Loss : 0.5282868146896362 Accuracy : 81.93000030517578 %
Epoch : 1 Batch : 40 Loss : 0.2851073741912842 Accuracy : 85.22000122070312 %
Epoch : 1 Batch : 50 Loss : 0.2913413345813751 Accuracy : 85.44000244140625 %
Epoch : 1 Batch : 60 Loss : 0.3766421973705292 Accuracy : 86.41999816894531 %
Epoch : 1 Batch : 70 Loss : 0.3795894682407379 Accuracy : 87.61000061035156 %
Epoch : 1 Batch : 80 Loss : 0.3694334030151367 Accuracy : 88.69999694824219 %


KeyboardInterrupt: ignored